# Projekt zaliczeniowy - cz 1 - sciagniecie danych z internetu 

Ponizszy kod sluzyl do pozyskania danych z OtoDom.
Obecnie po zmianach na stronie OtoDom kod juz nie dziala.

In [3]:
import scrapy
from scrapy.crawler import CrawlerProcess
from bs4 import BeautifulSoup
import requests

class OtodomBot(scrapy.Spider):
    name = "otodom_scraper"
    def start_requests(self):
        base = "https://www.otodom.pl/sprzedaz/mieszkanie/katowice/?nrAdsPerPage=72&page="
        for page_number in range(1, pages_all+1):
            url = base+str(page_number)
            yield scrapy.Request(url=url, callback=self.parse)
    def parse(self, response):
        urls = response.css("div.offer-item-details > header > h3 > a::attr(href)").extract()
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse_one_page)
    def parse_one_page(self, response):
        price = response.css("#root > div > article > header > div.css-15n3v4t-AdHeader-className > div.css-7ryazv-AdHeader-className::text").extract_first()
        details = response.css("#root > div > article > div.css-k68zts-AdPageTemplate-className > div.css-b8lhu7-AdPageTemplate-className > section.section-overview > div.css-1v9yl6n-AdOverview-className > ul > li").extract()
        if price is not None:
            record = {"price": price,
                  "details": details,
                  "url": response.request.url}
            real_estates.append(record)
real_estates = [] 
pages_all = 2000  
process = CrawlerProcess()
process.crawl(OtodomBot)
process.start()

2019-04-15 21:31:34 [scrapy.utils.log] INFO: Scrapy 1.6.0 started (bot: scrapybot)
2019-04-15 21:31:34 [scrapy.utils.log] INFO: Versions: lxml 4.2.5.0, libxml2 2.9.8, cssselect 1.0.3, parsel 1.5.1, w3lib 1.20.0, Twisted 18.7.0, Python 3.7.2 (default, Dec 29 2018, 06:19:36) - [GCC 7.3.0], pyOpenSSL 18.0.0 (OpenSSL 1.1.1b  26 Feb 2019), cryptography 2.6.1, Platform Linux-4.15.0-47-generic-x86_64-with-debian-buster-sid
2019-04-15 21:31:34 [scrapy.crawler] INFO: Overridden settings: {}
2019-04-15 21:31:34 [scrapy.extensions.telnet] INFO: Telnet Password: de8e002841f6c88c
2019-04-15 21:31:34 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2019-04-15 21:31:34 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.Downlo

ReactorNotRestartable: 

In [3]:
real_estates

NameError: name 'real_estates' is not defined

In [1]:
import json

In [2]:
with open(r'mieszkania.json', mode ='w') as file:
    file.write(str(real_estates))

NameError: name 'real_estates' is not defined

In [2]:
import pandas as pd
import json
with open('mieszkania.json', "r") as json_file:  
    data = json_file.read().replace("'", '"')
    with open('mieszkania_v2.json', "w") as fixed_json_file:
        fixed_json_file.write(data)

dane = ""
with open('mieszkania_v2.json', 'r') as json_file:  
    dane = json.load(json_file)

dane

[{'price': '299 000 zł',
  'details': ['<li>Powierzchnia: <strong>40 m²</strong></li>',
   '<li>Liczba pokoi: <strong>2</strong></li>',
   '<li>Rynek: <strong>pierwotny</strong></li>',
   '<li>Rodzaj zabudowy: <strong>apartamentowiec</strong></li>',
   '<li>Piętro: <strong>2</strong></li>',
   '<li>Liczba pięter: <strong>8</strong></li>',
   '<li>Materiał budynku: <strong>pustak</strong></li>',
   '<li>Okna: <strong>plastikowe</strong></li>',
   '<li>Ogrzewanie: <strong>miejskie</strong></li>',
   '<li>Rok budowy: <strong>2019</strong></li>',
   '<li>Stan wykończenia: <strong>do zamieszkania</strong></li>',
   '<li>Forma własności: <strong>pełna własność</strong></li>'],
  'url': 'https://www.otodom.pl/oferta/kup-i-zamieszkaj-przy-dolinie-prawie-gotowe-ID3Yj8M.html#4b2b486214'},
 {'price': '275 709 zł',
  'details': ['<li>Powierzchnia: <strong>48,37 m²</strong></li>',
   '<li>Liczba pokoi: <strong>3</strong></li>',
   '<li>Rynek: <strong>pierwotny</strong></li>',
   '<li>Rodzaj zabudow

In [8]:
import re

import unidecode


key_reg = re.compile('<li>([^\:]+):\s+<strong>([^>]*)</strong></li>')

def remove_pl_chars(mystr):
    clean = unidecode.unidecode(mystr.lower())
    clean = clean.replace(' ', '_')
    clean = clean.replace('_m2', '')
    clean = clean.replace('/m2', '')
    clean = clean.replace('_zl', '')
    clean = clean.replace(',', '.')
    return clean

all = []

for item in dane:
    new_item = {}
    new_item['cena'] = remove_pl_chars(item['price'])
    new_item['url'] = item['url']
    for detail in item["details"]:
        result = key_reg.match(detail)
        key = remove_pl_chars(result[1])
        value = remove_pl_chars(result[2])  
        #print(f"{key} => {value}")
        new_item[key] = value
    all.append(new_item)

df = pd.DataFrame(all)
df[["cena", "powierzchnia", "czynsz", "cena_za_od"]] = df[["cena", "powierzchnia", "czynsz", "cena_za_od"]].replace("_", '').replace('', 0).astype(float)

In [9]:
df

,cena,cena_za_od,czynsz,dostepne_od,forma_wlasnosci,liczba_kondygnacji,liczba_pieter,liczba_pokoi,material_budynku,metraz_do,...,okna,pietro,powierzchnia,rodzaj_zabudowy,rok_budowy,rynek,stan_inwestycji,stan_wykonczenia,typ_oferowanych_nieruchomosci,url
0,299000.0,NaN,NaN,NaN,pelna_wlasnosc,NaN,8,2,pustak,NaN,...,plastikowe,2,40.00,apartamentowiec,2019,pierwotny,NaN,do_zamieszkania,NaN,https://www.otodom.pl/oferta/kup-i-zamieszkaj-...
1,275709.0,NaN,NaN,NaN,NaN,NaN,10,3,NaN,NaN,...,plastikowe,1,48.37,apartamentowiec,NaN,pierwotny,NaN,do_wykonczenia,NaN,https://www.otodom.pl/oferta/nowa-inwestycja-m...
2,518891.0,NaN,NaN,NaN,NaN,NaN,10,4,zelbet,NaN,...,plastikowe,2,79.22,blok,NaN,pierwotny,NaN,NaN,NaN,https://www.otodom.pl/oferta/nowe-tysiaclecie-...
3,255000.0,NaN,NaN,NaN,pelna_wlasnosc,NaN,NaN,3,NaN,NaN,...,plastikowe,10,50.00,apartamentowiec,2019,pierwotny,NaN,do_wykonczenia,NaN,https://www.otodom.pl/oferta/narozny-apartamen...
4,290457.0,NaN,NaN,NaN,NaN,NaN,10,3,NaN,NaN,...,plastikowe,1,49.23,apartamentowiec,NaN,pierwotny,NaN,do_wykonczenia,NaN,https://www.otodom.pl/oferta/nowa-inwestycja-m...
5,270000.0,NaN,NaN,,pelna_wlasnosc,NaN,3,3,NaN,NaN,...,NaN,3,45.00,NaN,NaN,wtorny,NaN,NaN,NaN,https://www.otodom.pl/oferta/zarabiajace-miesz...
6,693528.0,NaN,NaN,NaN,pelna_wlasnosc,NaN,6,4,NaN,NaN,...,aluminiowe,6,78.81,apartamentowiec,NaN,pierwotny,NaN,do_wykonczenia,NaN,https://www.otodom.pl/oferta/uwaga-lux-apartam...
7,627200.0,NaN,NaN,NaN,pelna_wlasnosc,NaN,6,3,NaN,NaN,...,aluminiowe,6,64.00,apartamentowiec,NaN,pierwotny,NaN,do_wykonczenia,NaN,https://www.otodom.pl/oferta/zacznij-cieszyc-s...
8,337994.0,NaN,180.0,NaN,pelna_wlasnosc,NaN,4,3,NaN,NaN,...,NaN,4,61.12,apartamentowiec,2019,pierwotny,NaN,do_wykonczenia,NaN,https://www.otodom.pl/oferta/nowe-rozkladowe-m...
9,636300.0,NaN,NaN,NaN,pelna_wlasnosc,NaN,6,4,NaN,NaN,...,aluminiowe,6,70.70,apartamentowiec,NaN,pierwotny,NaN,do_wykonczenia,NaN,https://www.otodom.pl/oferta/lux-apartamentowi...


In [ ]:
df.to_csv("mieszkania_df.csv") # zrzucenie z df do csv